In [1]:
import torch
from heat_solver import HeatSolver
from inverse_solver import InverseSolver
from utils import create_conductivity_field, sine_source, sine_cosine_source, relative_rmse, r2_score
from utils import SimpleSigma, SigmoidSigma

from run import get_boundary_conditions

import numpy as np

## Omega experiment

In [ ]:
omega_arr = np.array([0.1, 0.3, 1.0, 3.0, 10.0]) * 2 * np.pi
M = 10
T = 1.0
device = 'cpu'
max_sigma = 5
alpha = 0.0001
sigma_0 = 1.0  # Initial guess
lr = 1e-1
noise_level = 0.05
max_iters = 5000
tol = 1e-1
pattern = 'linear'

source_func = lambda x, y, t: 10 * sine_cosine_source(x, y, t, omega)
sigma_gt = create_conductivity_field(M, pattern=pattern, device=device)

In [4]:
relative_mse_arr = []

for i, omega in enumerate(omega_arr):
    print(f"Optimizing sigma for omega={omega:.3f}")

    # Generate boundary observations with noise
    u_b_gt = get_boundary_conditions(sigma_gt, source_func, T, max_sigma, device=device)
    u_b = (1 + noise_level * torch.randn_like(u_b_gt)) * u_b_gt

    sigma_module = SimpleSigma(M, sigma_0)

    # One solver run to balance losses
    sigma = sigma_module()
    solver = HeatSolver(M, source_func, device)
    _, u_b_history, u_history = solver(sigma, T, max_sigma=max_sigma)

    reg_loss = solver.h**2 * (sigma - sigma_0).square().sum().item()
    data_loss = solver.h * solver.tau * (u_b_history - u_b).square().sum().item()
    alpha = 0.1 * data_loss / reg_loss

    # Inverse solver
    inverse_solver = InverseSolver(
        sigma_module,
        u_b_gt=u_b,
        source_func=source_func,
        M=M,
        T=T,
        n_steps=u_b.shape[0] - 1,
        alpha=alpha,
        sigma_0=sigma_0,
        device=device
    )

    final_sigma, total_loss_history, boundary_loss_history, regularization_loss_history = inverse_solver.solve(
        max_iters=max_iters,
        tol=tol,
        print_info=False
    )


    mse_arr.append(r2_score(final_sigma, sigma_gt))

Optimizing sigma for omega=0.628


  0%|          | 1/5000 [00:04<5:34:26,  4.01s/it]

Iter 0: Loss = 0.170519


  0%|          | 2/5000 [00:08<5:51:05,  4.21s/it]

Iter 1: Loss = 0.169875


...

Iter 28: Loss = 0.154624


  1%|          | 30/5000 [02:09<6:05:48,  4.42s/it]

Iter 29: Loss = 0.154140


  1%|          | 30/5000 [02:14<6:10:31,  4.47s/it]


KeyboardInterrupt: 

In [ ]:
plt.plot(omega_arr, mse_arr)
plt.xlabel(r'$\omega$')
plt.ylabel(r'$R^2$ score')
plt.xscale('log')
plt.show()

# $\alpha$ experiment

In [ ]:
M = 10
T = 1.0
device = 'cpu'
max_sigma = 5
omega = 2 * np.pi
alpha_arr = np.array([1e-4, 1e-3, 1e-2, 1e-1, 1])
sigma_0 = 1.0  # Initial guess
lr = 1e-1
noise_level = 0.05
max_iters = 5000
tol = 1e-1
pattern = 'linear'

source_func = lambda x, y, t: 10 * sine_cosine_source(x, y, t, omega)
sigma_gt = create_conductivity_field(M, pattern=pattern, device=device)

In [ ]:
relative_mse_arr = []

for i, alpha in enumerate(alpha_arr):
    print(f"Optimizing sigma for alpha={alpha:.3f}")

    # Generate boundary observations with noise
    u_b_gt = get_boundary_conditions(sigma_gt, source_func, T, max_sigma, device=device)
    u_b = (1 + noise_level * torch.randn_like(u_b_gt)) * u_b_gt

    sigma_module = SimpleSigma(M, sigma_0)

    # Inverse solver
    inverse_solver = InverseSolver(
        sigma_module,
        u_b_gt=u_b,
        source_func=source_func,
        M=M,
        T=T,
        n_steps=u_b.shape[0] - 1,
        alpha=alpha,
        sigma_0=sigma_0,
        device=device
    )

    final_sigma, total_loss_history, boundary_loss_history, regularization_loss_history = inverse_solver.solve(
        max_iters=max_iters,
        tol=tol,
        print_info=False
    )


    mse_arr.append(r2_score(final_sigma, sigma_gt))

In [ ]:
plt.plot(alpha_arr, mse_arr)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$R^2$ score')
plt.xscale('log')
plt.show()

# Transformations of $\sigma$ experiment

In [ ]:
M = 10
T = 1.0
device = 'cpu'
max_sigma = 5
omega = 2 * np.pi
alpha_arr = 0.0001
sigma_0 = 1.0  # Initial guess
lr = 1e-1
noise_level = 0.05
max_iters = 5000
tol = 1e-1
pattern = 'linear'

source_func = lambda x, y, t: 10 * sine_cosine_source(x, y, t, omega)
sigma_gt = create_conductivity_field(M, pattern=pattern, device=device)

sigma_modules = [SimpleSigma, SigmoidSigma]

In [ ]:
relative_mse_arr = []

for i, sigma_module in enumerate(sigma_modules):
    print(f"Optimizing sigma {i+1}/2")

    # Generate boundary observations with noise
    u_b_gt = get_boundary_conditions(sigma_gt, source_func, T, max_sigma, device=device)
    u_b = (1 + noise_level * torch.randn_like(u_b_gt)) * u_b_gt

    # Inverse solver
    inverse_solver = InverseSolver(
        sigma_module,
        u_b_gt=u_b,
        source_func=source_func,
        M=M,
        T=T,
        n_steps=u_b.shape[0] - 1,
        alpha=alpha,
        sigma_0=sigma_0,
        device=device
    )

    final_sigma, total_loss_history, boundary_loss_history, regularization_loss_history = inverse_solver.solve(
        max_iters=max_iters,
        tol=tol,
        print_info=False
    )


    mse_arr.append(r2_score(final_sigma, sigma_gt))

# Noise level experiment

In [ ]:
M = 10
T = 1.0
device = 'cpu'
max_sigma = 5
omega = 2 * np.pi
alpha_arr = 0.0001
sigma_0 = 1.0  # Initial guess
lr = 1e-1
noise_level_arr = np.array([1e-3, 0.01, 0.05, 0.1])
max_iters = 5000
tol = 1e-1
pattern = 'linear'

source_func = lambda x, y, t: 10 * sine_cosine_source(x, y, t, omega)
sigma_gt = create_conductivity_field(M, pattern=pattern, device=device)

In [ ]:
relative_mse_arr = []

for i, noise_level in enumerate(noise_level_arr):
    print(f"Optimizing sigma for noise_level={noise_level:.3f}")

    # Generate boundary observations with noise
    u_b_gt = get_boundary_conditions(sigma_gt, source_func, T, max_sigma, device=device)
    u_b = (1 + noise_level * torch.randn_like(u_b_gt)) * u_b_gt

    sigma_module = SimpleSigma(M, sigma_0)

    # Inverse solver
    inverse_solver = InverseSolver(
        sigma_module,
        u_b_gt=u_b,
        source_func=source_func,
        M=M,
        T=T,
        n_steps=u_b.shape[0] - 1,
        alpha=alpha,
        sigma_0=sigma_0,
        device=device
    )

    final_sigma, total_loss_history, boundary_loss_history, regularization_loss_history = inverse_solver.solve(
        max_iters=max_iters,
        tol=tol,
        print_info=False
    )


    mse_arr.append(r2_score(final_sigma, sigma_gt))

In [ ]:
plt.plot(noise_level_arr, mse_arr)
plt.xlabel('Noise level')
plt.ylabel(r'$R^2$ score')
plt.show()